# Assignment: Information Retrieval (IR)

## Preparations
* Put all your imports, and path constants in the next cells

In [1]:
!pip install whoosh
!pip install pytrec_eval
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 KB 6.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pytrec_eval: filename=pytrec_eval-0.5-cp38-cp38-linux_x86_64.whl size=301882 sha256=9eebf37ebe5e0170e5c5e3ba08e1ee6384c09179b8c628c03b7fa5fe55c7c15e
  Stored in directory: /root/.cache/pip/wheels/65/5a/e7/115104a6d6ebacea7cbd7434fdba04bd06b6756820994a326e
Successfully built pytrec_eval
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=6977fb5ab85ff08dfc9e0cf893f05eb6b8384b2ffa9f0e708f3ce1233b78304c
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f

In [2]:
import wget
wget.download("https://github.com/MIE1513HS-2022/course-datasets/raw/main/government.zip", "government.zip")

'government.zip'

In [3]:
!unzip government.zip

Archive:  government.zip
   creating: government/
  inflating: government/topics-with-full-descriptions.txt  
  inflating: government/gov.topics   
  inflating: government/gov.qrels    
   creating: government/documents/
   creating: government/documents/61/
  inflating: government/documents/61/G00-61-2800209  
  inflating: government/documents/61/G00-61-1192048  
  inflating: government/documents/61/G00-61-1118212  
  inflating: government/documents/61/G00-61-0749882  
  inflating: government/documents/61/G00-61-2230501  
  inflating: government/documents/61/G00-61-0680698  
  inflating: government/documents/61/G00-61-0551387  
  inflating: government/documents/61/G00-61-2575433  
  inflating: government/documents/61/G00-61-0469713  
  inflating: government/documents/61/G00-61-0280746  
  inflating: government/documents/61/G00-61-2574316  
  inflating: government/documents/61/G00-61-3933997  
  inflating: government/documents/61/G00-61-3290635  
  inflating: government/documents/61/G0

In [4]:
# imports
# Put all your imports here
from whoosh import index, writing, qparser,scoring
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import *
from whoosh.qparser import QueryParser
import os.path
from pathlib import Path
import tempfile
import subprocess
import pytrec_eval
import wget
import abc
from abc import abstractmethod
from whoosh.analysis import Filter

In [5]:
import nltk
nltk.download('omw-1.4')
from nltk.stem import *
# download required resources
nltk.download("wordnet")

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [6]:
class IRSystem(metaclass=abc.ABCMeta):
    """
    Abstract class which is inherited by other IR system
    """

    def __init__(self, data_dir):
        # DON'T change the following names,topic_file, qrels_file, document_dir, file_list
        self.topic_file = os.path.join(data_dir, "gov.topics")
        self.qrels_file = os.path.join(data_dir, "gov.qrels")
        self.document_dir = os.path.join(data_dir, "documents") 
        self.file_list = [str(filePath) for filePath in Path(self.document_dir).glob("**/*") if filePath.is_file()]

        self.create_index()
        self.add_files()
        self.create_parser_searcher()

        

    @abstractmethod
    def create_index(self):
        pass

    @abstractmethod
    def add_files(self):
        pass

    @abstractmethod
    def create_parser_searcher(self):
        pass

    @abstractmethod
    def perform_search(self, topic_phrase):
        pass

    @staticmethod
    def post_process_score(score):
        return score

    @staticmethod
    def print_trec_eval_result(results):
        if not results:
            print('empty results')
            return

        def print_line(name, scope, num):
            print('{:25s}{:8s}{:.4f}'.format(name, scope, num))

        for query_id, query_measures in results.items():
            for measure, value in query_measures.items():
                if measure == "runid":
                    continue
                print_line(measure, query_id, value)

        for measure in query_measures.keys():
            if measure == "runid":
                continue
            print_line(
                measure,
                'all',
                pytrec_eval.compute_aggregated_measure(
                    measure,
                    [query_measures[measure]
                     for query_measures in results.values()]))
            
    def print_rel_name(self, q_id):
        with open(self.topic_file, "r") as tf:
            topics = tf.read().splitlines()
        for topic in topics:
            topic_id, topic_phrase = tuple(topic.split(" ", 1))
            if topic_id == q_id:
                print("---------------------------Topic_id and Topic_phrase----------------------------------")
                print(topic_id, topic_phrase)
                 # get search result
                topic_results = self.perform_search(topic_phrase)
                print("---------------------------Return documents----------------------------------")
                for (docnum, result) in enumerate(topic_results):
                    score = topic_results.score(docnum)
                    score = self.post_process_score(score)
                    print("%s Q0 %s %d %lf test" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
                print("---------------------------Relevant documents----------------------------------")
                with open(self.qrels_file, 'r') as f_qrel:
                    qrels = f_qrel.readlines()
                    for i in qrels:
                        qid, _, doc, rel = i.rstrip().split(" ")
                        if qid == q_id and rel == "1":
                            print(i.rstrip())

    def py_trec_eval(self):
        # Load topic file - a list of topics(search phrases) used for evalutation
        with open(self.topic_file, "r") as tf:
            topics = tf.read().splitlines()

            # create an output file to which we'll write our results
        temp_output_file = tempfile.mkstemp()[1]
        with open(temp_output_file, "w") as outputTRECFile:
            # for each evaluated topic:
            # build a query and record the results in the file in TREC_EVAL format
            for topic in topics:
                topic_id, topic_phrase = tuple(topic.split(" ", 1))
                # get search result
                topic_results = self.perform_search(topic_phrase)
                # format the result
                for (docnum, result) in enumerate(topic_results):
                    score = topic_results.score(docnum)
                    outputTRECFile.write(
                        "%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))

        with open(self.qrels_file, 'r') as f_qrel:
            qrel = pytrec_eval.parse_qrel(f_qrel)

        with open(temp_output_file, 'r') as f_run:
            run = pytrec_eval.parse_run(f_run)

        evaluator = pytrec_eval.RelevanceEvaluator(
            qrel, pytrec_eval.supported_measures)

        results = evaluator.evaluate(run)

        self.print_trec_eval_result(results)


In [7]:
# Dont change this! Use it as-is in your code
# This filter will run for both the index and the query
class CustomFilter(Filter):
    is_morph = True
    def __init__(self, filterFunc, *args, **kwargs):
        self.customFilter = filterFunc
        self.args = args
        self.kwargs = kwargs
    def __eq__(self):
        return (other
                and self.__class__ is other.__class__)
    def __call__(self, tokens):
        for t in tokens:
            if t.mode == 'query': # if called by query parser
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t
            else: # == 'index' if called by indexer
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t

## Question 1
Provide your text answers in the following two markdown cells

### Q1 (a): Provide answer to Q1 (a) here [markdown cell]

Mean average precision (map)


### Q1 (b): Provide answer to Q1 (b) here [markdown cell]

Mean average precision average the precision over all relevant documents, non-interpolated. It is the average of the precision value obtained after each relevant document is retrieved. The measure is not an average of the precision at standard recall levels. \
For the government websites, we need the information retrieved over all relevent documents as much as possible, and there are many government websites. Moreover, the mean average precision average rewards the IR system that retrieve relevant documents quickly. Using this measure would have an accurate measure of the performance of the IR system.

## Question 2

### Q2 (a): Write your code below

**1. The auto-grader will extract and use the following variables, DON'T change the their names:**

      self.topic_file  
      self.qrels_file  
      self.document_dir   
      self.file_list  
      self.index_sys  
      self.query_parser  
      self.searcher   



**2. DON'T change the names of the already defined funtions**  
**3. DON'T change the py_trec_eval function**  
**4. DON'T change the class names including CustomFilter, IRSystem, IRQ2, IRQ3, IRQ4**  
**5. DON'T change the CustomFilter class and DON'T create any new custom filter class that is used to define Whoosh schema**

In [8]:
class IRQ2(IRSystem):
    def create_index(self):
        """
        INPUT:
            None
        OUTPUT:
            None
        
        NOTE: Please update self.index_sys which should have type whoosh.index.FileIndex
        """

        #change
        schema = Schema(file_path = ID(stored=True),
                      file_content = TEXT(analyzer = RegexTokenizer()))
        indexDir = tempfile.mkdtemp()

        # DON't change the name of 'index_sys'
        #self.index_sys = None
        self.index_sys = index.create_in(indexDir, schema)

    def add_files(self):
        """
        INPUT:
            None
        OUTPUT:
            None
        
        NOTE: Add buffer to self.index_sys
        """
        writer = writing.BufferedWriter(self.index_sys, period=None, limit=1000)

        try:
        # write each file to index
            for docNum, filePath in enumerate(self.file_list):
                with open(filePath, "r", encoding="utf-8") as f:
                    fileContent = f.read()
                    writer.add_document(file_path = filePath,
                                    file_content = fileContent)

                # print status every 1000 documents
                if (docNum+1) % 1000 == 0:
                    print("already indexed:", docNum+1)
            print("done indexing.")

        finally:
        # close the index
            writer.close()
        #pass

    def create_parser_searcher(self):
        """
        INPUT:
            None
        OUTPUT:
            None
        
        NOTE: Please update self.query_parser and self.searcher which should have type whoosh.qparser.default.QueryParser and whoosh.searching.Searcher respectively 
        """
         # DON't change the names of 'query_parser' and 'searcher'
        self.query_parser = QueryParser("file_content", schema=self.index_sys.schema)
        self.searcher = self.index_sys.searcher()

    def perform_search(self, topic_phrase):
        """
        INPUT:
            topic_phrase: string
        OUTPUT:
            topic_results: whoosh.searching.Results
        
        NOTE: Utilize self.query_parser and self.searcher to calculate the result for topic_phrase
        """
        query = self.query_parser.parse(topic_phrase)
        topic_results = self.searcher.search(query, limit = None)
        return topic_results

In [9]:
q2 = IRQ2("government")

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [10]:
q2.py_trec_eval()

num_q                    1       1.0000
num_ret                  1       1.0000
num_rel                  1       5.0000
num_rel_ret              1       0.0000
map                      1       0.0000
gm_map                   1       -11.5129
Rprec                    1       0.0000
bpref                    1       0.0000
recip_rank               1       0.0000
iprec_at_recall_0.00     1       0.0000
iprec_at_recall_0.10     1       0.0000
iprec_at_recall_0.20     1       0.0000
iprec_at_recall_0.30     1       0.0000
iprec_at_recall_0.40     1       0.0000
iprec_at_recall_0.50     1       0.0000
iprec_at_recall_0.60     1       0.0000
iprec_at_recall_0.70     1       0.0000
iprec_at_recall_0.80     1       0.0000
iprec_at_recall_0.90     1       0.0000
iprec_at_recall_1.00     1       0.0000
P_5                      1       0.0000
P_10                     1       0.0000
P_15                     1       0.0000
P_20                     1       0.0000
P_30                     1       0.000

In [11]:
q2.print_rel_name('1')

---------------------------Topic_id and Topic_phrase----------------------------------
1 mining gold silver coal
---------------------------Return documents----------------------------------
1 Q0 G00-90-0342721 0 26.645398 test
---------------------------Relevant documents----------------------------------
1 0 G00-00-1006224 1
1 0 G00-02-0901987 1
1 0 G00-03-1898526 1
1 0 G00-10-3730888 1
1 0 G00-10-3849661 1


### Q2 (b): Provide answer to Q2 (b) here [markdown cell]

The mean average precision for the retrieved documents is 0.1971.

### Q2 (c): Provide answer to Q2(c) here [markdown cell]

The system performed well on query with id = 18  and 24 , with `map = 1`. \
The system performed very bad on query 1,2,6,7,9,16,28, with `map = 0`. 

## Question 3

### Q3 (a): Provide answer to Q3 (a) here [markdown cell]

For the topic id = 10, the relevent document is only `G00-04-0412407`, but we have \
1. Documents `G00-35-1295262`, `G00-84-0557790`, `G00-05-0768168`, `G00-57-3463009`, `G00-15-2555140` were highly ranked, but not relevent, are false positive.
2. Document `G00-04-0412407` should have been highly ranked, is false negative. \

Here we choose  `G00-35-1295262` as the false positive case, and `G00-04-0412407` as the false negative case for error analysis.

In [30]:
#q2.print_rel_name('10')

---------------------------Topic_id and Topic_phrase----------------------------------
10 Physical Fitness
---------------------------Return documents----------------------------------
10 Q0 G00-35-1295262 0 21.260071 test
10 Q0 G00-84-0557790 1 20.170494 test
10 Q0 G00-05-0768168 2 19.314199 test
10 Q0 G00-57-3463009 3 19.231220 test
10 Q0 G00-15-2555140 4 17.956802 test
10 Q0 G00-04-0412407 5 17.331525 test
10 Q0 G00-04-0014570 6 16.839847 test
10 Q0 G00-71-1781567 7 15.970703 test
10 Q0 G00-27-1222379 8 15.254768 test
10 Q0 G00-95-0287843 9 13.962222 test
10 Q0 G00-22-0689568 10 13.471601 test
10 Q0 G00-86-4022163 11 7.933353 test
10 Q0 G00-33-2347653 12 4.187383 test
10 Q0 G00-82-3144058 13 4.078393 test
10 Q0 G00-24-3625720 14 3.869019 test
10 Q0 G00-96-2877140 15 3.226150 test
---------------------------Relevant documents----------------------------------
10 0 G00-04-0412407 1


In the document `G00-35-1295262`, all of the appearance of physical fitness is in the form "physical fitness award", the document is related to the award, rather than physical fitness, so although it is irrelevant, but is is ranked highly. \
However, in the document, `G00-04-0412407`, the content of the website is all about physical fitness, but the website contains fewer "physical fitness", so the although it is relevant but it is not ranked highly.

I propose a possible solution is to add `LowercaseFilter()` and `WordNetLemmatizer()` filter on the `RegexTokenizer()`.

Reason: `RegexTokenizer()` is not case -sensitive, we first need to transfer the words into lower case using `LowercaseFilter()`, `WordNetLemmatizer()` links words into semantic relations including synonyms, hyponyms, and meronyms, to rule out the situation described as above.

### Q3 (b): Write your code below

**1. The auto-grader will extract and use the following variables, DON'T change the their names:**

      self.topic_file  
      self.qrels_file  
      self.document_dir   
      self.file_list  
      self.index_sys  
      self.query_parser  
      self.searcher   



**2. DON'T change the names of the already defined funtions**  
**3. DON'T change the py_trec_eval function**  
**4. DON'T change the class names including CustomFilter, IRSystem, IRQ2, IRQ3, IRQ4**  
**5. DON'T change the CustomFilter class and DON'T create any new custom filter class that is used to define Whoosh schema**

In [13]:
class IRQ3(IRSystem):
    def create_index(self):
        """
        INPUT:
            None
        OUTPUT:
            None
        
        NOTE: Please update self.index_sys which should have type whoosh.index.FileIndex
        """
        # DON't change the name of 'index_sys'
        updated_analyzer = RegexTokenizer() | LowercaseFilter() |CustomFilter(WordNetLemmatizer().lemmatize)

        schema = Schema(file_path = ID(stored=True),
                        file_content = TEXT(analyzer = updated_analyzer))
        indexDir = tempfile.mkdtemp()

        # self.index_sys = None
        self.index_sys = index.create_in(indexDir, schema)

        

    def add_files(self):
        """
        INPUT:
            None
        OUTPUT:
            None
        
        NOTE: Add buffer to self.index_sys
        """
        # pass
        writer = writing.BufferedWriter(self.index_sys, period=None, limit=1000)

        try:
        # write each file to index
            for docNum, filePath in enumerate(self.file_list):
                with open(filePath, "r", encoding="utf-8") as f:
                    fileContent = f.read()
                    writer.add_document(file_path = filePath,
                                    file_content = fileContent)

                # print status every 1000 documents
                if (docNum+1) % 1000 == 0:
                    print("already indexed:", docNum+1)
            print("done indexing.")

        finally:
        # close the index
            writer.close()
        #pass

    def create_parser_searcher(self):
        """
        INPUT:
            None
        OUTPUT:
            None
        
        NOTE: Please update self.query_parser and self.searcher which should have type whoosh.qparser.default.QueryParser and whoosh.searching.Searcher respectively 
        """
         # DON't change the names of 'query_parser' and 'searcher'
        # self.query_parser = None
        # self.searcher = None

        self.query_parser = QueryParser("file_content", schema=self.index_sys.schema)
        self.searcher = self.index_sys.searcher()

    def perform_search(self, topic_phrase):
        """
        INPUT:
            topic_phrase: string
        OUTPUT:
            topicResults: whoosh.searching.Results
        
        NOTE: Utilize self.query_parser and self.searcher to calculate the result for topic_phrase
        """
        # topicResults = None
        # return topicResults
        query = self.query_parser.parse(topic_phrase)
        topic_results = self.searcher.search(query, limit = None)

        return topic_results

In [14]:
q3 = IRQ3("government")

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [15]:
q3.py_trec_eval()

num_q                    1       1.0000
num_ret                  1       3.0000
num_rel                  1       5.0000
num_rel_ret              1       0.0000
map                      1       0.0000
gm_map                   1       -11.5129
Rprec                    1       0.0000
bpref                    1       0.0000
recip_rank               1       0.0000
iprec_at_recall_0.00     1       0.0000
iprec_at_recall_0.10     1       0.0000
iprec_at_recall_0.20     1       0.0000
iprec_at_recall_0.30     1       0.0000
iprec_at_recall_0.40     1       0.0000
iprec_at_recall_0.50     1       0.0000
iprec_at_recall_0.60     1       0.0000
iprec_at_recall_0.70     1       0.0000
iprec_at_recall_0.80     1       0.0000
iprec_at_recall_0.90     1       0.0000
iprec_at_recall_1.00     1       0.0000
P_5                      1       0.0000
P_10                     1       0.0000
P_15                     1       0.0000
P_20                     1       0.0000
P_30                     1       0.000

In [16]:
q3.print_rel_name('10')

---------------------------Topic_id and Topic_phrase----------------------------------
10 Physical Fitness
---------------------------Return documents----------------------------------
10 Q0 G00-35-1295262 0 18.826408 test
10 Q0 G00-27-1222379 1 17.870720 test
10 Q0 G00-04-0412407 2 17.708215 test
10 Q0 G00-84-0557790 3 17.531416 test
10 Q0 G00-57-3463009 4 17.502458 test
10 Q0 G00-05-0768168 5 17.146758 test
10 Q0 G00-15-2555140 6 16.873183 test
10 Q0 G00-04-0014570 7 14.591965 test
10 Q0 G00-71-1781567 8 14.332499 test
10 Q0 G00-95-0287843 9 13.453742 test
10 Q0 G00-22-0689568 10 11.673332 test
10 Q0 G00-12-0024883 11 10.326700 test
10 Q0 G00-44-2026102 12 8.952308 test
10 Q0 G00-86-4022163 13 6.874362 test
10 Q0 G00-45-4032177 14 6.762220 test
10 Q0 G00-63-3078014 15 4.561949 test
10 Q0 G00-26-3218156 16 4.561949 test
10 Q0 G00-33-2347653 17 4.360768 test
10 Q0 G00-24-3625720 18 4.046143 test
10 Q0 G00-82-3144058 19 3.467417 test
10 Q0 G00-70-2787853 20 3.292930 test
10 Q0 G00-96-28

### Q3 (c): Provide answer to Q3 (c) here [markdown cell]

Modification: add `LowercaseFilter()` and `WordNetLemmatizer()` filter on the `RegexTokenizer()`. \

The MAP is now increased from 0.1971 to 0.3243.  \

The ranking score for false positive `G00-35-1295262` is 18.826408, which is decrease significantly (21.260071 before modification), and the ranking score false negative case `G00-04-0412407` is 17.708215 now, which is higher than before. 

### Q3 (d): Provide answer to Q3 (d) here [markdown cell]

The MAP is now increased from 0.1971 to 0.3243, so the modification improves overall.

### Q3 (e): Provide answer to Q3 (e) here [markdown cell]

Querys for topic 2,4,9,10,14,19,22,28 got better; the query for topic 26 got worse.

### Q3 (f): Provide answer to Q3 (f) here [markdown cell]

Since the overall map increased from 0.1971 to 0.3243, so I would say this means my modification is good. The score for false positive decrease significantly and the score for false negative increase. However, the IR system so far still regards websites with a directory contains the query word as more relevent than the website with content relevent to the topic of the query.

## Question 4


### Q4 (a): Provide answer to Q4 (a) here [markdown cell]

Through iterations, I found change the ranking function to TF-IDF lead to a low performance. 

Reason: TF-IDF rewards term frequency and penalizes document frequency, while BM25 goes beyond this to account for document length and term frequency saturation. 

### Q4 (b): Write your code below

**1. The auto-grader will extract and use the following variables, DON'T change the their names:**

      self.topic_file  
      self.qrels_file  
      self.document_dir   
      self.file_list  
      self.index_sys  
      self.query_parser  
      self.searcher   



**2. DON'T change the names of the already defined funtions**  
**3. DON'T change the py_trec_eval function**  
**4. DON'T change the class names including CustomFilter, IRSystem, IRQ2, IRQ3, IRQ4**  
**5. DON'T change the CustomFilter class and DON'T create any new custom filter class that is used to define Whoosh schema**

In [17]:
class IRQ4(IRSystem):
    def create_index(self):
        """
        INPUT:
            None
        OUTPUT:
            None
        
        NOTE: Please update self.index_sys which should have type whoosh.index.FileIndex
        """
        # DON't change the name of 'index_sys'

        # updated_analyzer = RegexTokenizer() | LowercaseFilter() |CustomFilter(WordNetLemmatizer().lemmatize)  # Q3
        # updated_analyzer = RegexTokenizer() | LowercaseFilter() | StemFilter() |CustomFilter(WordNetLemmatizer().lemmatize) # add StemFilter()
        # updated_analyzer = RegexTokenizer() | LowercaseFilter() | StopFilter() | StemFilter() |CustomFilter(WordNetLemmatizer().lemmatize) # add  StopFilter() | StemFilter() 
        updated_analyzer = RegexTokenizer() | LowercaseFilter() | StemFilter() |CustomFilter(LancasterStemmer().stem) # wordnet to lancaster
        # updated_analyzer = RegexTokenizer() | LowercaseFilter() | StemFilter() |CustomFilter(PorterStemmer().stem) #lancaster to porterstemmer
        schema = Schema(file_path = ID(stored=True),
                        file_content = TEXT(analyzer = updated_analyzer))
        indexDir = tempfile.mkdtemp()

        self.index_sys = index.create_in(indexDir, schema)


    def add_files(self):
        """
        INPUT:
            None
        OUTPUT:
            None
        
        NOTE: Add buffer to self.index_sys
        """
        # pass
        writer = writing.BufferedWriter(self.index_sys, period=None, limit=1000)

        try:
        # write each file to index
            for docNum, filePath in enumerate(self.file_list):
                with open(filePath, "r", encoding="utf-8") as f:
                    fileContent = f.read()
                    writer.add_document(file_path = filePath,
                                    file_content = fileContent)

                # print status every 1000 documents
                if (docNum+1) % 1000 == 0:
                    print("already indexed:", docNum+1)
            print("done indexing.")

        finally:
        # close the index
            writer.close()


    def create_parser_searcher(self):
        """
        INPUT:
            None
        OUTPUT:
            None
        
        NOTE: Please update self.query_parser and self.searcher which should have type whoosh.qparser.default.QueryParser and whoosh.searching.Searcher respectively 
        """
         # DON't change the names of 'query_parser' and 'searcher'
        # self.query_parser = None
        # self.searcher = None

        self.query_parser = QueryParser("file_content", schema=self.index_sys.schema)
        # self.searcher = self.index_sys.searcher(weighting = scoring.TF_IDF)
        self.searcher = self.index_sys.searcher()

    def perform_search(self, topic_phrase):
        """
        INPUT:
            topic_phrase: string
        OUTPUT:
            topicResults: whoosh.searching.Results
        
        NOTE: Utilize self.query_parser and self.searcher to calculate the result for topic_phrase
        """
        # topicResults = None
        # return topicResults

        query = self.query_parser.parse(topic_phrase)
        topic_results = self.searcher.search(query, limit = None)

        return topic_results

        

In [18]:
#updated_analyzer = RegexTokenizer() | LowercaseFilter() | StemFilter() |CustomFilter(WordNetLemmatizer().lemmatize) # add StemFilter()

In [19]:
#updated_analyzer = RegexTokenizer() | LowercaseFilter() | StopFilter() | StemFilter() |CustomFilter(WordNetLemmatizer().lemmatize) # add StopFilter() | StemFilter() 

In [20]:
#updated_analyzer = RegexTokenizer() | LowercaseFilter() | StemFilter() |CustomFilter(LancasterStemmer().stem) # wordnet to lancaster

In [21]:
#updated_analyzer = RegexTokenizer() | LowercaseFilter() | StemFilter() |CustomFilter(PorterStemmer().stem) #lancaster to porterstemmer

In [22]:
q4 = IRQ4("government")

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [23]:
q4.py_trec_eval()

num_q                    1       1.0000
num_ret                  1       3.0000
num_rel                  1       5.0000
num_rel_ret              1       0.0000
map                      1       0.0000
gm_map                   1       -11.5129
Rprec                    1       0.0000
bpref                    1       0.0000
recip_rank               1       0.0000
iprec_at_recall_0.00     1       0.0000
iprec_at_recall_0.10     1       0.0000
iprec_at_recall_0.20     1       0.0000
iprec_at_recall_0.30     1       0.0000
iprec_at_recall_0.40     1       0.0000
iprec_at_recall_0.50     1       0.0000
iprec_at_recall_0.60     1       0.0000
iprec_at_recall_0.70     1       0.0000
iprec_at_recall_0.80     1       0.0000
iprec_at_recall_0.90     1       0.0000
iprec_at_recall_1.00     1       0.0000
P_5                      1       0.0000
P_10                     1       0.0000
P_15                     1       0.0000
P_20                     1       0.0000
P_30                     1       0.000

In [24]:
q4.print_rel_name('1')

---------------------------Topic_id and Topic_phrase----------------------------------
1 mining gold silver coal
---------------------------Return documents----------------------------------
1 Q0 G00-90-0342721 0 22.332160 test
1 Q0 G00-55-3817584 1 13.705527 test
1 Q0 G00-69-2353421 2 7.088952 test
---------------------------Relevant documents----------------------------------
1 0 G00-00-1006224 1
1 0 G00-02-0901987 1
1 0 G00-03-1898526 1
1 0 G00-10-3730888 1
1 0 G00-10-3849661 1


### Q4 (b): Provide answer to Q4 (b) here [markdown cell]

1. In Q3, we have analyzer: \
`RegexTokenizer() | LowercaseFilter() | StemFilter() |CustomFilter(WordNetLemmatizer().lemmatize)` with map = 0.3243
2. To ignore common stop words, add `StopFilter()` to analyzer:\
`RegexTokenizer() | LowercaseFilter() | StopFilter() | StemFilter() |CustomFilter(WordNetLemmatizer().lemmatize)` with map = 0.3363.
3. This time, change the ranking function to TF-IDF:  \
`RegexTokenizer() | LowercaseFilter() | StemFilter() |CustomFilter(WordNetLemmatizer().lemmatize)`, we have map = 0.1619, the performance does not increase. 
4. Now I change the lemmatizer to stemmer:  \
`RegexTokenizer() | LowercaseFilter() | StemFilter() |CustomFilter(LancasterStemmer().stem)` with map = 0.3579, this time the analyzer perform better.
5. Change to another stemmer:  \
`RegexTokenizer() | LowercaseFilter() | StemFilter() |CustomFilter(PorterStemmer().stem)` with map = 0.3372, this time the analyzer perform worse.  \
so we have analyzer `RegexTokenizer() | LowercaseFilter() | StemFilter() |CustomFilter(LancasterStemmer().stem)` with ranking function BM25 perform best.


### Q4 (c): Provide answer to Q4 (c) here [markdown cell]

There is an improvement in performance with analyzer.   \
 `RegexTokenizer() | LowercaseFilter() | StemFilter() |CustomFilter(LancasterStemmer().stem)`, the map is 0.3579;  \
In Q3, with analyzer `RegexTokenizer() | LowercaseFilter() |CustomFilter
(WordNetLemmatizer().lemmatize)` we have map = 0.3243

## Validation

#### Run the following cells to make sure your code returns the correct value types

In [25]:
from whoosh.index import FileIndex
from whoosh.qparser import QueryParser
from whoosh.searching import Searcher
import os.path

### Q2 Validation

In [26]:
assert(isinstance(q2.index_sys, FileIndex)), "Index Type"
assert(isinstance(q2.query_parser, QueryParser)), "Query Parser Type"
assert(isinstance(q2.searcher, Searcher)), "Searcher Type"
print("Q2 Types Validated")

Q2 Types Validated


### Q3 Validation

In [27]:
assert(isinstance(q3.index_sys, FileIndex)), "Index Type"
assert(isinstance(q3.query_parser, QueryParser)), "Query Parser Type"
assert(isinstance(q3.searcher, Searcher)), "Searcher Type"
print("Q3 Types Validated")

Q3 Types Validated


### Q4 Validation

In [28]:
assert(isinstance(q4.index_sys, FileIndex)), "Index Type"
assert(isinstance(q4.query_parser, QueryParser)), "Query Parser Type"
assert(isinstance(q4.searcher, Searcher)), "Searcher Type"
print("Q4 Types Validated")

Q4 Types Validated
